Item-Based Collaborative Filtering
========================

In [446]:
import pandas as pd
import numpy as np
#from StringIO import StringIO

from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')#give a nice white grid background for visualizations
#plt.switch_backend('GTKAgg')
%matplotlib inline



In [447]:
r_cols = ['user_id', 'game_id', 'rating']
reviews = pd.read_csv('C://Users/pmlef/Documents/Python_work/Python_for_Data_Analysis/Recommender Data/boardgame-elite-users.csv', sep=',', names=r_cols, dtype='str', skiprows=1, usecols=range(3), encoding="utf-8")

my_reviews = pd.read_csv('C://Users/pmlef/Documents/Python_work/Python_for_Data_Analysis/Recommender Data/my-ratings.csv', sep=',', names=r_cols, dtype='str', skiprows=1, usecols=range(3), encoding="utf-8")

#In the future should probably use a different seperator because ',' is too common
m_cols = ['game_id', 'title']
games = pd.read_csv('C://Users/pmlef/Documents/Python_work/Python_for_Data_Analysis/Recommender Data/boardgame-titles.csv', sep=',', names=m_cols, dtype='str',skiprows=1,usecols=range(2), encoding="utf-8")

#ratings = pd.merge(games, reviews)

reviews.info(),print('     '), games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59293 entries, 0 to 59292
Data columns (total 3 columns):
user_id    59293 non-null object
game_id    59293 non-null object
rating     59293 non-null object
dtypes: object(3)
memory usage: 1.4+ MB
     
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 2 columns):
game_id    402 non-null object
title      402 non-null object
dtypes: object(2)
memory usage: 6.4+ KB


(None, None, None)

In [448]:
my_reviews

,user_id,game_id,rating
0,9999999,13,8
1,9999999,74,5
2,9999999,98,4
3,9999999,171,7
4,9999999,181,7
5,9999999,188,4
6,9999999,320,4
7,9999999,463,9
8,9999999,891,2
9,9999999,926,7


In [449]:
reviews[:10]

,user_id,game_id,rating
0,272,9216,1
1,272,9217,1
2,272,39938,1
3,272,15363,5
4,272,24068,1
5,272,70149,1
6,272,43015,1
7,272,107529,1
8,272,10,1
9,272,12,1


In [450]:
games[:10]

,game_id,title
0,13004,The Downfall of Pompeii
1,66188,Fresco
2,503,Through the Desert
3,66690,Dominion: Prosperity
4,150,PitchCar
5,181,Risk
6,21059,Arkham Horror: Dunwich Horror Expansion
7,90419,Airlines Europe
8,31627,Ticket to Ride: Nordic Countries
9,41114,The Resistance


In [451]:
reviews = reviews.append(my_reviews)

In [452]:
reviews

,user_id,game_id,rating
0,272,9216,1
1,272,9217,1
2,272,39938,1
3,272,15363,5
4,272,24068,1
5,272,70149,1
6,272,43015,1
7,272,107529,1
8,272,10,1
9,272,12,1


In [453]:
ratings = pd.merge(reviews, games, on='game_id')

In [454]:
ratings.head()

,user_id,game_id,rating,title
0,272,9216,1,Goa
1,388,9216,10,Goa
2,430,9216,8,Goa
3,2044,9216,8,Goa
4,3080,9216,8,Goa


In [455]:
#Checking titles after merge
ratings['title'].unique()

array(['Goa', 'Saint Petersburg', 'Carson City', 'Nexus Ops',
       'Shadow Hunters', 'Ora et Labora', 'Hansa Teutonica',
       'Kingdom Builder', 'Elfenland', 'Ra', 'Catan',
       'Space Hulk (third edition)',
       'Age of Empires III: The Age of Discovery', 'RoboRally', 'Samurai',
       'Chinatown', 'Vegas Showdown', 'Kingsburg',
       'A Game of Thrones: The Board Game (Second Edition)', 'Agricola',
       'Ubongo', 'Acquire', 'Saboteur', 'War of the Ring (Second Edition)',
       'Poker', 'Flash Point: Fire Rescue', 'Cosmic Encounter',
       "Can't Stop", 'Dungeons & Dragons: Castle Ravenloft Board Game',
       'The Princes of Florence', "Liar's Dice", 'Medici', 'Le Havre',
       'Brass: Lancashire', 'Friday', 'Ricochet Robots', 'Space Alert',
       'Tikal', 'Letters from Whitechapel', 'Finca', 'Mr. Jack Pocket',
       '1960: The Making of the President', 'Trains',
       'Robinson Crusoe: Adventures on the Cursed Island',
       'Descent: Journeys in the Dark', 'Claust

In [456]:
ratings.describe() #top is the most common (i.e mode)

,user_id,game_id,rating,title
count,59324,59324,59324,59324
unique,200,402,317,402
top,187094,68448,7,7 Wonders
freq,383,197,13170,197


Ratings descriptive stats

In [457]:
ratings['rating'].describe()

count     59324
unique      317
top           7
freq      13170
Name: rating, dtype: object

In [458]:
ratings['rating'].median()

7.0

In [459]:
ratings['rating'].mode()

0    7
dtype: object

USER ID descriptive stats

In [460]:
ratings['user_id'].describe()

count      59324
unique       200
top       187094
freq         383
Name: user_id, dtype: object

Ratings descriptive stats

In [461]:
ratings['game_id'].describe()

count     59324
unique      402
top       68448
freq        197
Name: game_id, dtype: object

Setting Ratings to Numeric

In [462]:
ratings.rating = pd.to_numeric(ratings.rating, errors='coerce').fillna(0).astype(np.int64)

Groupby 

In [463]:
gb_title = ratings['rating'].groupby(ratings['title'])

In [464]:
#Titles with average greater than 7
gb_title_avg=gb_title.mean()
gb_title_avg[gb_title_avg>7].sort_values().tail()

title
Puerto Rico                                      7.803109
Dominion: Prosperity                             7.840909
Pandemic Legacy: Season 1                        7.848276
Terra Mystica                                    7.857143
Through the Ages: A New Story of Civilization    8.000000
Name: rating, dtype: float64

In [465]:
#titles with median greater than 7
gb_title_med=gb_title.median()
gb_title_avg[gb_title_avg>7].sort_values().tail()

title
Puerto Rico                                      7.803109
Dominion: Prosperity                             7.840909
Pandemic Legacy: Season 1                        7.848276
Terra Mystica                                    7.857143
Through the Ages: A New Story of Civilization    8.000000
Name: rating, dtype: float64

In [466]:
# Number of Reviews for each title
gb_title_cnt=gb_title.count()
gb_title_cnt.sort_values().tail()

title
The Castles of Burgundy    195
Lords of Waterdeep         196
Citadels                   196
Ticket to Ride             196
7 Wonders                  197
Name: rating, dtype: int64

In [467]:
Ratings_Cats_Meow=ratings.copy()

In [468]:
Ratings_Cats_Meow[:3]

,user_id,game_id,rating,title
0,272,9216,1,Goa
1,388,9216,10,Goa
2,430,9216,8,Goa


Groupby rating less than 3

In [469]:
gb_cats_meow_one=Ratings_Cats_Meow[Ratings_Cats_Meow['rating']<4]
gb_cats_meow_one.sort_values('rating')

,user_id,game_id,rating,title
0,272,9216,1,Goa
23848,44871,258,1,Fluxx
23851,54797,258,1,Fluxx
23863,74377,258,1,Fluxx
23871,86991,258,1,Fluxx
23874,96794,258,1,Fluxx
23882,101885,258,1,Fluxx
23899,132526,258,1,Fluxx
23926,166007,258,1,Fluxx
23940,272,21763,1,Mr. Jack


In [470]:
gb_cats_meow_one = gb_cats_meow_one.pivot_table(index=['user_id'],columns=['title'],values='rating')
gb_cats_meow_one.head()

title,1960: The Making of the President,6 nimmt!,7 Wonders,7 Wonders: Leaders,A Feast for Odin,A Few Acres of Snow,A Game of Thrones (first edition),A Game of Thrones: The Board Game (Second Edition),Above and Below,Abyss,...,Wizard,XCOM: The Board Game,YINSH,Yahtzee,Yspahan,Zombicide,Zombicide: Black Plague,Zombie Dice,Zombies!!!,Zooloretto
user_id,,,,,,,,,,,,,,,,,,,,,
100055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100383,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Groupby rating greater than 6

In [471]:
gb_cats_meow_two=Ratings_Cats_Meow[Ratings_Cats_Meow['rating']>6]
gb_cats_meow_two

,user_id,game_id,rating,title
1,388,9216,10,Goa
2,430,9216,8,Goa
3,2044,9216,8,Goa
4,3080,9216,8,Goa
5,3256,9216,9,Goa
6,5038,9216,7,Goa
7,5217,9216,9,Goa
8,5480,9216,8,Goa
9,7100,9216,8,Goa
10,9363,9216,8,Goa


In [472]:
gb_cats_meow_two = gb_cats_meow_two.pivot_table(index=['user_id'],columns=['title'],values='rating')
gb_cats_meow_two.head()

title,1960: The Making of the President,6 nimmt!,7 Wonders,7 Wonders Duel,7 Wonders: Cities,7 Wonders: Leaders,A Feast for Odin,A Few Acres of Snow,A Game of Thrones (first edition),A Game of Thrones: The Board Game (Second Edition),...,Wizard,XCOM: The Board Game,YINSH,Yahtzee,Yspahan,Zombicide,Zombicide: Black Plague,Zombie Dice,Zombies!!!,Zooloretto
user_id,,,,,,,,,,,,,,,,,,,,,
100055,NaN,9.0,10.0,9.0,NaN,NaN,9.0,NaN,NaN,NaN,...,8.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN
100174,10.0,NaN,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,NaN,NaN
100383,8.0,9.0,7.0,7.0,7.0,7.0,NaN,7.0,NaN,NaN,...,NaN,NaN,7.0,NaN,8.0,NaN,NaN,NaN,NaN,7.0
100790,NaN,NaN,7.0,NaN,NaN,NaN,7.0,NaN,7.0,NaN,...,7.0,NaN,NaN,NaN,7.0,7.0,NaN,NaN,NaN,7.0
101462,NaN,NaN,NaN,7.0,NaN,NaN,NaN,7.0,7.0,NaN,...,NaN,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [473]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
#userRatings = userRatings.dropna(axis=1,how='all')
userRatings

title,1960: The Making of the President,6 nimmt!,7 Wonders,7 Wonders Duel,7 Wonders: Cities,7 Wonders: Leaders,A Feast for Odin,A Few Acres of Snow,A Game of Thrones (first edition),A Game of Thrones: The Board Game (Second Edition),...,Wizard,XCOM: The Board Game,YINSH,Yahtzee,Yspahan,Zombicide,Zombicide: Black Plague,Zombie Dice,Zombies!!!,Zooloretto
user_id,,,,,,,,,,,,,,,,,,,,,
100055,NaN,9.0,10.0,9.0,NaN,NaN,9.0,NaN,NaN,NaN,...,8.0,NaN,NaN,7.0,6.0,NaN,NaN,NaN,NaN,6.0
100174,10.0,NaN,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,NaN,NaN
100383,8.0,9.0,7.0,7.0,7.0,7.0,NaN,7.0,2.0,6.0,...,6.0,NaN,7.0,5.0,8.0,5.0,NaN,NaN,NaN,7.0
100790,NaN,5.0,7.0,NaN,NaN,NaN,7.0,NaN,7.0,NaN,...,7.0,NaN,NaN,NaN,7.0,7.0,NaN,5.0,NaN,7.0
101462,6.0,5.0,5.0,7.0,NaN,5.0,NaN,7.0,7.0,NaN,...,6.0,8.0,8.0,4.0,6.0,NaN,NaN,NaN,NaN,5.0
101583,6.0,7.0,6.0,7.0,NaN,6.0,NaN,9.0,9.0,9.0,...,NaN,6.0,9.0,5.0,6.0,NaN,NaN,4.0,7.0,6.0
101885,NaN,8.0,4.0,NaN,NaN,NaN,NaN,7.0,4.0,NaN,...,6.0,NaN,7.0,6.0,5.0,NaN,NaN,NaN,NaN,5.0
103398,6.0,8.0,6.0,6.0,NaN,6.0,7.0,5.0,6.0,7.0,...,6.0,NaN,3.0,6.0,6.0,NaN,NaN,NaN,NaN,7.0
104693,5.0,8.0,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,7.0,5.0,5.0,6.0,6.0,6.0,3.0,7.0


In [474]:
corrMatrix = userRatings.corr()
corrMatrix.head()

title,1960: The Making of the President,6 nimmt!,7 Wonders,7 Wonders Duel,7 Wonders: Cities,7 Wonders: Leaders,A Feast for Odin,A Few Acres of Snow,A Game of Thrones (first edition),A Game of Thrones: The Board Game (Second Edition),...,Wizard,XCOM: The Board Game,YINSH,Yahtzee,Yspahan,Zombicide,Zombicide: Black Plague,Zombie Dice,Zombies!!!,Zooloretto
title,,,,,,,,,,,,,,,,,,,,,
1960: The Making of the President,1.000000,0.303162,0.467949,0.313209,0.373976,0.532885,0.220747,0.420741,0.342503,0.294846,...,0.268085,0.247562,0.343572,0.209730,0.394444,0.407531,0.393965,0.420390,0.324247,0.322086
6 nimmt!,0.303162,1.000000,0.239066,0.124045,0.074838,0.114512,0.165932,0.261182,0.144023,0.165384,...,0.421091,0.087594,0.335494,0.261912,0.249098,0.191577,-0.010439,0.353160,0.176690,0.345537
7 Wonders,0.467949,0.239066,1.000000,0.564393,0.613150,0.644046,0.235882,0.423036,0.360871,0.426540,...,0.271215,0.249229,0.258064,0.373459,0.480003,0.422234,0.274371,0.224224,0.212718,0.377307
7 Wonders Duel,0.313209,0.124045,0.564393,1.000000,0.547575,0.496099,0.352776,0.466634,0.263157,0.339881,...,0.255359,0.442255,0.303554,0.148925,0.449497,0.311363,0.317211,0.298021,0.108776,0.306511
7 Wonders: Cities,0.373976,0.074838,0.613150,0.547575,1.000000,0.774936,0.282140,0.504302,0.329498,0.358119,...,0.065039,0.509350,0.410555,0.196589,0.340791,0.329012,0.516650,0.228505,0.178352,0.346320


In [475]:
corrMatrix = userRatings.corr(method='pearson', min_periods=100)
corrMatrix.head()

title,1960: The Making of the President,6 nimmt!,7 Wonders,7 Wonders Duel,7 Wonders: Cities,7 Wonders: Leaders,A Feast for Odin,A Few Acres of Snow,A Game of Thrones (first edition),A Game of Thrones: The Board Game (Second Edition),...,Wizard,XCOM: The Board Game,YINSH,Yahtzee,Yspahan,Zombicide,Zombicide: Black Plague,Zombie Dice,Zombies!!!,Zooloretto
title,,,,,,,,,,,,,,,,,,,,,
1960: The Making of the President,1.000000,0.303162,0.467949,0.313209,NaN,0.532885,NaN,0.420741,0.342503,0.294846,...,NaN,NaN,0.343572,NaN,0.394444,NaN,NaN,NaN,NaN,0.322086
6 nimmt!,0.303162,1.000000,0.239066,0.124045,NaN,0.114512,0.165932,0.261182,0.144023,0.165384,...,0.421091,0.087594,0.335494,0.261912,0.249098,0.191577,NaN,0.353160,NaN,0.345537
7 Wonders,0.467949,0.239066,1.000000,0.564393,0.61315,0.644046,0.235882,0.423036,0.360871,0.426540,...,0.271215,0.249229,0.258064,0.373459,0.480003,0.422234,NaN,0.224224,NaN,0.377307
7 Wonders Duel,0.313209,0.124045,0.564393,1.000000,NaN,0.496099,0.352776,0.466634,0.263157,0.339881,...,0.255359,0.442255,0.303554,0.148925,0.449497,0.311363,NaN,0.298021,NaN,0.306511
7 Wonders: Cities,NaN,NaN,0.613150,NaN,1.00000,0.774936,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [476]:
myRatings = userRatings.iloc[199].dropna()
myRatings

title
Apples to Apples                            5.0
Axis & Allies                               4.0
Backgammon                                  4.0
Cards Against Humanity                      4.0
Catan                                       8.0
Catan: 5-6 Player Extension                 7.0
Catan: Cities & Knights                     7.0
Checkers                                    3.0
Chess                                       7.0
Clue                                        6.0
Cranium                                     2.0
Dixit                                       6.0
Dixit Quest                                 4.0
Forbidden Island                            5.0
Gloom                                       5.0
Go                                          4.0
Magic: The Gathering                        9.0
Monopoly                                    4.0
Munchkin                                    5.0
Mysterium                                   6.0
Pandemic                          

In [477]:
simCandidates = pd.Series()
for i in range(0, len(myRatings.index)):
    print("Adding sims for " + myRatings.index[i] + "...")
    # Retrieve similar games to the ones that I rated
    sims = corrMatrix[myRatings.index[i]].dropna()
    # Now scale its similarity by how well I rated this game
    sims = sims.map(lambda x: x * myRatings[i])
    # Add the score to the list of similar candidates
    simCandidates = simCandidates.append(sims)
    
#Glance at our results so far:
print("sorting...")
simCandidates.sort_values(inplace = True, ascending = False)
print(simCandidates.head(50))

Adding sims for Apples to Apples...
Adding sims for Axis & Allies...
Adding sims for Backgammon...
Adding sims for Cards Against Humanity...
Adding sims for Catan...
Adding sims for Catan: 5-6 Player Extension...
Adding sims for Catan: Cities & Knights...
Adding sims for Checkers...
Adding sims for Chess...
Adding sims for Clue...
Adding sims for Cranium...
Adding sims for Dixit...
Adding sims for Dixit Quest...
Adding sims for Forbidden Island...
Adding sims for Gloom...
Adding sims for Go...
Adding sims for Magic: The Gathering...
Adding sims for Monopoly...
Adding sims for Munchkin...
Adding sims for Mysterium...
Adding sims for Pandemic...
Adding sims for Risk...
Adding sims for Scrabble...
Adding sims for Shogun...
Adding sims for Sid Meier's Civilization: The Board Game...
Adding sims for Star Realms...
Adding sims for Stratego...
Adding sims for Taboo...
Adding sims for UNO...
Adding sims for Yahtzee...
Adding sims for Zombies!!!...
sorting...
Magic: The Gathering               

Getting Rid of Duplicates by Adding Ratings

In [478]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()

In [479]:
#simCandidates[:25].sort_values(ascending = False)

Recommended Games
==========

In [480]:
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(50)

Catan                                       40.561895
Forbidden Island                            38.186105
Pandemic                                    36.567545
Alhambra                                    35.817780
BANG!                                       35.637635
Ticket to Ride                              34.826735
Lords of Waterdeep                          34.547002
7 Wonders                                   34.204296
Citadels                                    34.081389
Stone Age                                   33.912424
Ticket to Ride: Europe                      33.574992
Machi Koro                                  32.816850
Eclipse                                     32.771196
Small World                                 32.485579
King of Tokyo                               32.153575
Kingsburg                                   32.073386
San Juan                                    31.969320
Can't Stop                                  31.815028
Thurn and Taxis             

In [481]:
#myRatings

In [482]:
#simCandidates.index

In [483]:
#myRatings.index

Removing **My Ratings** From the Recommendations

In [484]:
filteredSims = simCandidates.drop(myRatings.index,errors='ignore')
filteredSims.head(30)

Alhambra                             35.817780
BANG!                                35.637635
Ticket to Ride                       34.826735
Lords of Waterdeep                   34.547002
7 Wonders                            34.204296
Citadels                             34.081389
Stone Age                            33.912424
Ticket to Ride: Europe               33.574992
Machi Koro                           32.816850
Eclipse                              32.771196
Small World                          32.485579
King of Tokyo                        32.153575
Kingsburg                            32.073386
San Juan                             31.969320
Can't Stop                           31.815028
Thurn and Taxis                      31.658374
The Pillars of the Earth             31.594522
Dominion: Intrigue                   31.485775
Jaipur                               31.307653
Five Tribes                          30.640951
Lost Cities                          30.597555
Tikal        